In [246]:
import pandas as pd
import regex as re
import plotly
from plotly import graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_white'

## Background

The webscraper was used to retrieve hearing data for Judges Paula Skahan and Mark Ward during the week of 1/10/22 - 1/14/22. Data was exported to two CSV files: parties contains information about the defendants while charges contains charges they're facing. The data is shown below for the two individual datasets and for the merged data joined on *Case Number*.

In [6]:
parties = pd.read_csv('party_data.csv').drop('Unnamed: 0', axis=1)
charges = pd.read_csv('charges_data.csv').drop('Unnamed: 0', axis=1)

In [9]:
parties.head()

,Case Number,Judge,Courtroom,Hearing Date,Defendant,DOB,Race
0,C1909455,Division I - Judge Paula Skahan,Division 1,2022-01-10,"RUZICHA, TRAVIS",02/22/2000,White
1,C1802145,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR",11/12/1989,Black
2,C1809571,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR LOVE",11/12/1989,Black
3,C1910146,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR LOVE",11/12/1989,Black
4,C2101702,Division I - Judge Paula Skahan,Division 1,2022-01-10,"WEDDINGTON, PARNELL THOMAS",03/30/1962,Black


In [25]:
charges.head()

,Case Number,Charge Number,Description,Statute,Level,Date
0,C1909455,1,RAPE OF A CHILD,39-13-522,Felony A,07/20/2019
1,C1909455,2,AGGRAVATED SEXUAL BATTERY,39-13-504,Felony B,07/20/2019
2,C1802145,1,AGGRAVATED BURGLARY-ACT IN CONCERT,39-14-403,Felony B,04/30/2017
3,C1802145,2,AGGRAVATED ASSAULT,39-13-102,Felony C,04/30/2017
4,C1802145,101,PETITION TO REVOKE SUSP SENTENCE,40-35-311,Civil-Petition,07/25/2019


In [33]:
merged_data = parties.join(charges.set_index('Case Number'), on='Case Number').reset_index(drop=True)
merged_data.head()

,Case Number,Judge,Courtroom,Hearing Date,Defendant,DOB,Race,Charge Number,Description,Statute,Level,Date
0,C1909455,Division I - Judge Paula Skahan,Division 1,2022-01-10,"RUZICHA, TRAVIS",02/22/2000,White,1,RAPE OF A CHILD,39-13-522,Felony A,07/20/2019
1,C1909455,Division I - Judge Paula Skahan,Division 1,2022-01-10,"RUZICHA, TRAVIS",02/22/2000,White,2,AGGRAVATED SEXUAL BATTERY,39-13-504,Felony B,07/20/2019
2,C1802145,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR",11/12/1989,Black,1,AGGRAVATED BURGLARY-ACT IN CONCERT,39-14-403,Felony B,04/30/2017
3,C1802145,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR",11/12/1989,Black,2,AGGRAVATED ASSAULT,39-13-102,Felony C,04/30/2017
4,C1802145,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR",11/12/1989,Black,101,PETITION TO REVOKE SUSP SENTENCE,40-35-311,Civil-Petition,07/25/2019


---

### Exploratory Analysis

Defendants can have multiple case numbers. I'm assuming these are all individual crimes carried out at different times and involving different charges. An example is shown below for the Defendant **Fair, Omar Love** who was seen on 01/10/2022 by Judge Skahan for 3 separate cases. For clarity, I'm only showing the first charge for each *Case Number*. Note that the dates for each of these charges is unique. With this in mind, when comparing the racial composition of cases between Judges Skahan and Ward, we can do so in a number of ways. We can treat every case as an individual data point, even if they share the same defendant, or we can look at only unique defendants seen in by each Judge. I'll proceed with the latter method as it makes sense to me to that defendants would be assigned to a judge and courtroom and have all of their cases heard by that judge.

In [39]:
merged_data[(merged_data.Defendant.str.contains('OMAR')) & (merged_data['Charge Number']==1)]

,Case Number,Judge,Courtroom,Hearing Date,Defendant,DOB,Race,Charge Number,Description,Statute,Level,Date
2,C1802145,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR",11/12/1989,Black,1,AGGRAVATED BURGLARY-ACT IN CONCERT,39-14-403,Felony B,04/30/2017
5,C1809571,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR LOVE",11/12/1989,Black,1,POSS OF CONT SUBSTANCE MARIJUANA,39-17-418,Misdemeanor A,09/17/2017
13,C1910146,Division I - Judge Paula Skahan,Division 1,2022-01-10,"FAIR, OMAR LOVE",11/12/1989,Black,1,UNLAWFUL CARRYING OR POSSESSION OF A WEAPON,39-17-1307,Felony B,04/20/2019


In [41]:
# drop duplicate defendants
unique_defendants = merged_data[['Judge', 'Defendant', 'Race']].drop_duplicates()
unique_defendants.head()

,Judge,Defendant,Race
0,Division I - Judge Paula Skahan,"RUZICHA, TRAVIS",White
2,Division I - Judge Paula Skahan,"FAIR, OMAR",Black
5,Division I - Judge Paula Skahan,"FAIR, OMAR LOVE",Black
20,Division I - Judge Paula Skahan,"WEDDINGTON, PARNELL THOMAS",Black
24,Division I - Judge Paula Skahan,"BARNES, KIA N",Black


In [42]:
unique_defendants[['Judge', 'Defendant']].groupby(['Judge']).count()

,Defendant
Judge,
Division I - Judge Paula Skahan,143
Division IX - Judge W. Mark Ward,172


During the week of 1/10/22 - 1/14/22, Judge Skahan proceeded over cases for 143 defendants while Judge Ward proceeded over cases for 172 unique defendants. The race of these defendants is broken down for each judge in the table below.

In [78]:
race_composition = unique_defendants.groupby(['Judge', 'Race']).count()
race_composition['Percentage of Group (Judge)'] = round(100*race_composition/race_composition.groupby('Judge').sum(), 2)
race_composition

Defendant  Percentage of Group (Judge)
Judge                            Race                                         
Division I - Judge Paula Skahan  Black        127                        88.81
                                 White         16                        11.19
Division IX - Judge W. Mark Ward Asian          1                         0.58
                                 Black        139                        80.81
                                 Other          1                         0.58
                                 White         31                        18.02

Judge Skahan saw a racial composition of 88.81% black and 11.19% white, while Judge Ward saw a composition of 80.81% black and 18.02% white.

In [82]:
merged_data.Statute.nunique()

94

There are a total of 94 unique statutes.

In [83]:
merged_data.Description.nunique()

165

There are a total of 165 unique charge descriptions.

In [86]:
unique_charges = merged_data[['Statute', 'Description']].drop_duplicates()
unique_charges

,Statute,Description
0,39-13-522,RAPE OF A CHILD
1,39-13-504,AGGRAVATED SEXUAL BATTERY
2,39-14-403,AGGRAVATED BURGLARY-ACT IN CONCERT
3,39-13-102,AGGRAVATED ASSAULT
4,40-35-311,PETITION TO REVOKE SUSP SENTENCE
...,...,...
1518,39-13-212,CRIMINALLY NEGLIGENT HOMICIDE
1526,39-13-304,CRIMINAL ATTEMPT-AGGRAVATED KIDNAPPING
1579,39-13-102,AGGRAVATED ASSAULT-RECKLESS-ACT CON
1594,39-14-505,AGGRAVATED CRIMINAL LITTERING


There are a total of 166 unique Statute and Description combinations.

In [105]:
stat_descriptions = unique_charges.groupby('Statute').filter(lambda x: x['Description'].count() > 1).sort_values('Statute')
stat_descriptions

,Statute,Description
1455,11-11-1111,MOTION FOR NEW TRIAL
1396,11-11-1111,MOTION TO CORRECT ILLEGAL SENTENCE
1373,11-11-1111,MOTION TO REOPEN PCR
643,11-11-1111,MOTION FOR FAST/SPEEDY/TRIAL/IND
435,11-11-1111,APPEAL
...,...,...
1090,55-10-101,LEAVE SCENE OF ACCIDENT INV DEATH
19,55-10-401,DRIVING U/INFLUENCE INTOX/DRUGS EQUAL OR GREAT...
6,55-10-401,DRIVING U/INFLUENCE INTOX/DRUGS
10,55-50-504,DRIVING WHILE LICENSE S/R/C


Below are the last 10 unique statute + descriptions. Looking at the first statute (39-17-434), we can see there are some data integrity issues. These two descriptions are identical except for one dash instead of a space between 'UPCS' and 'W/I'. Most of the statutes however do have more than one unique description.

In [107]:
stat_descriptions.iloc[-10:,:]

,Statute,Description
246,39-17-434,UPCS W/I METH
938,39-17-434,UPCS-W/I METH
585,40-35-311,VIOLATION OF PROBATION M
4,40-35-311,PETITION TO REVOKE SUSP SENTENCE
854,55-10-101,LEAVE SCENE OF ACCIDENT INV INJURY
1090,55-10-101,LEAVE SCENE OF ACCIDENT INV DEATH
19,55-10-401,DRIVING U/INFLUENCE INTOX/DRUGS EQUAL OR GREAT...
6,55-10-401,DRIVING U/INFLUENCE INTOX/DRUGS
10,55-50-504,DRIVING WHILE LICENSE S/R/C
248,55-50-504,DRIVING WHILE LICENSE S/R/C W/PRIOR


Below is the breakdown of the number of Charges for each type that each saw. For example, Judge Skahan oversaw 29 cases of *Aggravated Assault* and Judge Ward oversaw 7 cases of *Vehicular Homicide - Reckless*. The additional *Percentage of Group* column tells us for each charge, what percentage of all charges seen by each judge were of that charge. So the 29 *Aggravated Assault* charges seen by Judge Skahan comprised 4.68% of all charges they saw.

In [207]:
charge_counts = merged_data[['Judge', 'Description', 'Charge Number']].groupby(['Judge', 'Description']).count()
charge_counts['Percentage of Group (Judge)'] = round(100*charge_counts/charge_counts.groupby('Judge').sum(), 2)
charge_counts

Charge Number  \
Judge                            Description                                        
Division I - Judge Paula Skahan  AGG SEXUAL EXPLOITATION OF MINOR               3   
                                 AGGRAVATED ARSON                               1   
                                 AGGRAVATED ASSAULT                            29   
                                 AGGRAVATED ASSAULT - RECKLESS                  2   
                                 AGGRAVATED ASSAULT-ACT IN CONCERT             10   
...                                                                           ...   
Division IX - Judge W. Mark Ward VANDALISM $10000-60000                         2   
                                 VANDALISM $2500-$10000                         6   
                                 VEHICULAR ASSAULT                              2   
                                 VEHICULAR HOMICIDE - INTOXICATION              6   
                                 VEHICULAR HOMICIDE - RECKLESS                  7   

                                                                    Percentage of Group (Judge)  
Judge                            Description                                                     
Division I - Judge Paula Skahan  AGG SEXUAL EXPLOITATION OF MINOR                          0.48  
                                 AGGRAVATED ARSON                                          0.16  
                                 AGGRAVATED ASSAULT                                        4.68  
                                 AGGRAVATED ASSAULT - RECKLESS                             0.32  
                                 AGGRAVATED ASSAULT-ACT IN CONCERT                         1.61  
...                                                                                         ...  
Division IX - Judge W. Mark Ward VANDALISM $10000-60000                                    0.20  
                                 VANDALISM $2500-$10000                                    0.61  
                                 VEHICULAR ASSAULT                                         0.20  
                                 VEHICULAR HOMICIDE - INTOXICATION                         0.61  
                                 VEHICULAR HOMICIDE - RECKLESS                             0.72  

[230 rows x 2 columns]

This data is filtered down to the top 10 charges most seen by each Judge and summarized below.

In [275]:
top_charges = charge_counts.sort_values(['Judge', 'Charge Number'], ascending=False).groupby(['Judge']).head(10)
top_charges

Charge Number  \
Judge                            Description                                          
Division IX - Judge W. Mark Ward AGGRAVATED ASSAULT                              73   
                                 AGGRAVATED ROBBERY                              55   
                                 EMPLOY FIREARM W/I TO COMMIT FELONY             54   
                                 ILLEGAL REGISTRATION OR VOTING                  48   
                                 FIRST DEGREE MURDER                             41   
                                 PETITION TO REVOKE SUSP SENTENCE                28   
                                 CARJACKING                                      25   
                                 THEFT OF PROPERTY $10000-$60000                 25   
                                 TERMINATION OF DIVERSION                        24   
                                 POSS CONT SUB W/I MAN/DEL/SELL                  18   
Division I - Judge Paula Skahan  BURGLARY - MOTOR VEHICLE                        59   
                                 DRIVING U/INFLUENCE INTOX/DRUGS                 36   
                                 FIRST DEGREE MURDER                             36   
                                 THEFT OF MERCH LESS THAN $1000                  32   
                                 IDENTITY THEFT                                  30   
                                 AGGRAVATED ASSAULT                              29   
                                 AGGRAVATED CHILD ABUSE                          29   
                                 THEFT OF PROPERTY $1000 OR LESS                 22   
                                 EMPLOY FIREARM W/I TO COMMIT FELONY             18   
                                 PETITION TO REVOKE SUSP SENTENCE                17   

                                                                      Percentage of Group (Judge)  
Judge                            Description                                                       
Division IX - Judge W. Mark Ward AGGRAVATED ASSAULT                                          7.47  
                                 AGGRAVATED ROBBERY                                          5.63  
                                 EMPLOY FIREARM W/I TO COMMIT FELONY                         5.53  
                                 ILLEGAL REGISTRATION OR VOTING                              4.91  
                                 FIRST DEGREE MURDER                                         4.20  
                                 PETITION TO REVOKE SUSP SENTENCE                            2.87  
                                 CARJACKING                                                  2.56  
                                 THEFT OF PROPERTY $10000-$60000                             2.56  
                                 TERMINATION OF DIVERSION                                    2.46  
                                 POSS CONT SUB W/I MAN/DEL/SELL                              1.84  
Division I - Judge Paula Skahan  BURGLARY - MOTOR VEHICLE                                    9.52  
                                 DRIVING U/INFLUENCE INTOX/DRUGS                             5.81  
                                 FIRST DEGREE MURDER                                         5.81  
                                 THEFT OF MERCH LESS THAN $1000                              5.16  
                                 IDENTITY THEFT                                              4.84  
                                 AGGRAVATED ASSAULT                                          4.68  
                                 AGGRAVATED CHILD ABUSE                                      4.68  
                                 THEFT OF PROPERTY $1000 OR LESS                             3.55  
                                 EMPLOY FIREARM W/I TO COMMIT FELONY                         2.90  
                                 PETITION TO REVOKE SUS

In [276]:
top_charges = top_charges.reset_index()

data = []
for judge in top_charges.Judge.unique():
    data.append(
        go.Bar(name=judge, 
               x=top_charges[top_charges.Judge==judge].Description, 
               y=top_charges[top_charges.Judge==judge]['Charge Number']
              )
        )

fig = go.Figure(data)
fig.update_layout(title='Top 10 Charges by Judge', uniformtext_minsize=8, 
                  uniformtext_mode='show',barmode='group', xaxis_tickangle=-35,
                 yaxis={'title': 'Count'})

fig.show()

The same data is plotted in a bar chart above. Here's a brief summary of this figure:

1. The charge that appeared most in Judge Ward's courtroom was **Aggravated Assault** while the charge that appeared most in Judge Skahan's courtroom was **Burglary - Motor Vehicle**.
2. Judge Ward received more than double the number of **Aggravated Assault** cases compared to Judge Skahan (73 to 29).
3. Judge Ward's second highest seen charge was **Aggravated Robbery** for which Judge Skahan saw none.
4. Judge Skahan's second highest seen charge was tied between **First Degree Murder** and **Driving U/Influence Intox/Drugs**. However, Judge Ward still had more cases of **First Degree Murder**. 

This begs the question, which Judge presided over more serious crimes? To answer that I summarize the top charges for each judge in terms of the Level of the crime.

In [277]:
top_charges_by_level = top_charges.merge(charge_levels, left_on='Description', right_on='Description', how='left')
top_charges_by_level = top_charges_by_level[['Judge', 'Level', 'Charge Number']].groupby(['Judge', 'Level']).sum()
top_charges_by_level

Charge Number
Judge                            Level                          
Division I - Judge Paula Skahan  Civil-Petition               17
                                 Felony B                     29
                                 Felony C                     47
                                 Felony D                     30
                                 Felony E                     59
                                 Felony M                     36
                                 Misdemeanor A                90
Division IX - Judge W. Mark Ward Civil Ordinances             24
                                 Civil-Petition               28
                                 Felony B                     98
                                 Felony C                    170
                                 Felony D                     66
                                 Felony M                     41

In [278]:
top_charges_by_level = top_charges_by_level.reset_index()

data = []
for judge in top_charges.Judge.unique():
    
    data.append(
        go.Bar(name=judge, 
               x=top_charges_by_level[top_charges_by_level.Judge==judge]['Level'], 
               y=top_charges_by_level[top_charges_by_level.Judge==judge]['Charge Number']
              )
        )

fig = go.Figure(data)
fig.update_layout(title='Charge Level Counts', uniformtext_minsize=8, 
                  uniformtext_mode='show',barmode='group', xaxis_tickangle=-35,
                 yaxis={'title': 'Count'}, xaxis={'type': 'category'})

fig.show()

Above is a breakdown of the levels of the top 10 charges seen by each judge. Immediately visible is the difference in the distributions. Judge Ward saw many more Felony cases than Judge Skahan. Comparing the two most serious levels, Felony B and C, Judge Ward saw 268 of these cases while Skahan only saw 76. Judge Skahan saw 90 Misdemeanor cases while Ward saw none.

In [279]:
def level_class(x):
    if 'Felony' in x:
        return 'Felony'
    elif 'Mis' in x:
        return 'Misdemeanor'
    elif 'Civil' in x:
        return 'Civil'

top_charges_by_level['Level Class'] = top_charges_by_level.Level.apply(lambda x: level_class(x))

level_class = top_charges_by_level[['Judge', 'Level Class', 'Charge Number']].groupby(['Judge', 'Level Class']).sum()
level_class['Percentage of Group (Judge)'] = round(100*level_class/level_class.groupby('Judge').sum(), 2)
level_class

Charge Number  \
Judge                            Level Class                  
Division I - Judge Paula Skahan  Civil                   17   
                                 Felony                 201   
                                 Misdemeanor             90   
Division IX - Judge W. Mark Ward Civil                   52   
                                 Felony                 375   

                                              Percentage of Group (Judge)  
Judge                            Level Class                               
Division I - Judge Paula Skahan  Civil                               5.52  
                                 Felony                             65.26  
                                 Misdemeanor                        29.22  
Division IX - Judge W. Mark Ward Civil                              12.18  
                                 Felony                             87.82

- Class A Felony up to 60 years in prison and a fine of up to $50,000

- Class B Felony up to 30 years in prison and a fine of up to $25,000

- Class C Felony up to 15 years in prison and a fine of up to $10,000

- Class D Felony up to 12 years in prison and a fine of up to $5,000

- Class E Felony up to 6 years in prison and a fine of up to $3,000

- Class A Misdemeanor up to one year in prison and a fine of up to $2,500

- Class B Misdemeanor up to six months in jail and a fine of up to $500

- Class C Misdemeanor up to 30 days in jail and a fine of up to $50

source:https://www.midsouthcriminaldefense.com/blog/2014/january/what-is-the-difference-between-a-misdemeanor-and/#:~:text=There%20are%20five%20different%20classes,fine%20of%20up%20to%20%2410%2C000

This data can be summarized at a higher level by grouping all felonies, misdemeanors, and civil cases. We can see from the table above that Judge Ward saw 174 more felony cases than Judge Skahan. Judge Ward saw over 20% more felony cases than Skahan. The data shows that Judge ward presided over more significantly more serious cases with higher consequences than Skahan. The cell above summarizes the max consequences of each charge level. We can use this upper limit to calculate the difference in the potential total maximum sentencing between judges.

In [292]:
# mappings of the max prison sentences & fines for each level
prison_dict = {'Felony B': 30,
                'Felony C': 15,
                'Felony D': 12,
                'Felony E': 6,
                'Misdemeanor A': 1,
               'Civil Ordinances': 0,
               'Civil-Petition': 0,
               'Felony M': 0
              }

fine_dict = {'Felony B': 50000,
                'Felony C': 25000,
                'Felony D': 10000,
                'Felony E': 5000,
                'Misdemeanor A': 2500,
               'Civil Ordinances': 0,
               'Civil-Petition': 0,
               'Felony M': 0}

top_charges_by_level['Prison Sentence'] = top_charges_by_level.Level.replace(prison_dict)
top_charges_by_level['Fine'] = top_charges_by_level.Level.replace(fine_dict)
top_charges_by_level['Total Prison Sentence'] = top_charges_by_level['Charge Number'] * top_charges_by_level['Prison Sentence']
top_charges_by_level['Total Fine'] = top_charges_by_level['Charge Number'] * top_charges_by_level['Fine']
top_charges_by_level

,Judge,Level,Charge Number,Level Class,Prison Sentence,Fine,Total Prison Sentence,Total Fine
0,Division I - Judge Paula Skahan,Civil-Petition,17,Civil,0,0,0,0
1,Division I - Judge Paula Skahan,Felony B,29,Felony,30,50000,870,1450000
2,Division I - Judge Paula Skahan,Felony C,47,Felony,15,25000,705,1175000
3,Division I - Judge Paula Skahan,Felony D,30,Felony,12,10000,360,300000
4,Division I - Judge Paula Skahan,Felony E,59,Felony,6,5000,354,295000
5,Division I - Judge Paula Skahan,Felony M,36,Felony,0,0,0,0
6,Division I - Judge Paula Skahan,Misdemeanor A,90,Misdemeanor,1,2500,90,225000
7,Division IX - Judge W. Mark Ward,Civil Ordinances,24,Civil,0,0,0,0
8,Division IX - Judge W. Mark Ward,Civil-Petition,28,Civil,0,0,0,0
9,Division IX - Judge W. Mark Ward,Felony B,98,Felony,30,50000,2940,4900000


In [291]:
totals = top_charges_by_level[['Judge', 'Total Prison Sentence', 'Total Fine']].groupby(['Judge']).sum()
totals

,Total Prison Sentence,Total Fine
Judge,,
Division I - Judge Paula Skahan,2379,3445000
Division IX - Judge W. Mark Ward,6282,9810000


The table above shows that **Judge Ward** oversaw cases summing up to a potential total of **6,282 years** of prision time and **\\$9,810,000** in total fines. **Judge Skahan** oversaw cases summing up to a potential total of **2,379 years** of prision time and **\\$3,445,000** in total fines. A stark contrast.